# Stroke Prediction Dataset

### Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px
from plotly.subplots import make_subplots
py.init_notebook_mode(connected=True)

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

### Load data

In [ ]:
# caricamento dati
df = pd.read_csv('./healthcare-dataset-stroke-data.csv')
# dimensioni
nRow, nCol = df.shape
print('Le dimensioni del dataframe sono:')
print(nRow, nCol)

# nomi delle colonne
print(df.columns)

### Data cleaning

In [ ]:
# salvo una copia
dfOriginal = df.copy()

In [ ]:
df.info()

In [ ]:
# rinomino colonne
df.rename(columns = {'Residence_type':'residence_type'}, inplace = True)
columnNames = df.columns.tolist()
columnNames

In [ ]:
# trovo i valori NaN
for i in columnNames:
    print(str(i) + ' has: ' + str(df[i].isnull().values.sum()) + ' NaN values')
# BMI ha dei valori mancanti

In [ ]:
# rimuovo tutti i NaN presenti dal dataframe (in questo caso solo dalla colonna BMI)
df = df.dropna() # rimuove 201 righe
# rimuovo tutti i soggetti con età inferiore a 14 (età in cui è visibile il primo stroke = 1)
df = df[df.age >= 14] # rimuove 629 righe
# rimuovo il soggetto in cui non è specificato il genere
df = df[df.gender != 'Other'] # Rimuove 1 riga
df.reset_index(inplace=True, drop=True)
df.index

In [ ]:
# nuove dimensioni
nRow, nCol = df.shape
print('Le dimensioni del dataframe pulito sono:')
print(nRow, nCol)

### Prima analisi delle variabili numeriche

In [ ]:
# rimuovo le colonne degli ID e degli index per una visualizzazione grafica migliore
df_noID = df.copy().drop(columns=['id'])
df_noID.describe()

In [ ]:
df_noID.hist(bins = 10)
plt.tight_layout()
plt.show()

In [ ]:
# trovo i valori 'unique' per ogni colonna
for i in columnNames:
    if i == 'id' or i == 'index' or i == 'age' or i == 'bmi' or i == 'avg_glucose_level':
        pass
    else:
        print(str(i) + ' has: ' + str(pd.unique(df[i])) + ' unique values\n')

# Analisi sull'incidenza di ipertensione
Plotto la distribuzione del genere all'interno del dataset, la distribuzione dei soggetti che hanno riportato ipertensione e la distribuzione di quest'ultima all'interno delle due classi di genere.

Si può notare che il dataset include una leggera maggioranza di donne, l'incidenza di ipetensione è molto sbilanciata rispetto al dataset completo (solo 10.5% del totale) e che è leggermente più presente nel subset di soggetti maschili nonstante siano in numero minore.

Numero di soggetti che riportano ipertensione sul totale di soggetti (4279):

In [ ]:
df.query('hypertension==1').count()['hypertension']

Numero di soggetti di genere femminile che riportano ipertensione sul totale di soggetti femminili (2599):

In [ ]:
df[df.gender == 'Female'].query('hypertension==1').count()['hypertension']

Numero di soggetti di genere maschile che riportano ipertensione sul totale di soggetti maschili (1680):

In [ ]:
df[df.gender == 'Male'].query('hypertension==1').count()['hypertension']

In [ ]:
fig = make_subplots(
    rows=2, cols=2,subplot_titles=('<b>Distribution of <i>Genders</i></b>','<b>Distribution of <i>Hypertension</i></b>','<b><i>Hypertension</i> of Male Subjects</b>','<b><i>Hypertension</i> of Female Subjects</b>'),
    vertical_spacing=0.09,
    specs=[[{"type": "pie","colspan": 1}       ,{"type": "pie","colspan": 1}],
           [{"type": "pie","rowspan": 1}       ,{"type": "pie","rowspan": 1}],                                      
          ]
)

fig.add_trace(
    go.Pie(values=df.gender.value_counts().values,labels=['Donne','Uomini'],hole=0.5,pull=[0,0.02,0.5],marker_colors=['lightpink','lightblue'],textposition='inside'
          ),
    row=1, col=1
)

fig.add_trace(
    go.Pie(values=df.hypertension.value_counts().values,labels=['No Hypertension','Hypertension'],hole=0.5,pull=[0,0.02,0.5],marker_colors=['lightgreen','red'],textposition='inside'
          ),
    row=1, col=2
)

fig.add_trace(
    go.Pie(values=df.query('gender=="Male"').hypertension.value_counts().values,labels=['No Hypertension','Hypertension'],hole=0,pull=[0,0.02,0.5],marker_colors=['lightgreen','red'],textposition='inside'
          ),
    row=2, col=1
)

fig.add_trace(
    go.Pie(values=df.query('gender=="Female"').hypertension.value_counts().values,labels=['No Hypertension','Hypertension'],hole=0,pull=[0,0.02,0.5],marker_colors=['lightgreen','red'],textposition='inside'
          ),
    row=2, col=2
)

fig.update_layout(
    height=800,
    showlegend=True,
    title_text="<b>Gender and Hypertension<b>"
)

fig.show()

A questo punto plotto la distribuzione delle età dei soggetti del dataset all'interno di un istogramma evidenziando in rosso l'incidenza di ipertensione i base all'età.

In [ ]:
age_distr = px.histogram(
        df,x=df.age,color="hypertension",title='<b>Age distribution</b>',color_discrete_sequence=px.colors.qualitative.G10#Dark2
    )
age_distr.show()

L'età media dei soggetti è:

In [ ]:
df.age.mean()

L'età media dei soggetti che riportano ipertensione è:

In [ ]:
df.query('hypertension==1').mean()['age'].round(2)

### Categorie in base all'indice di massa corporea (BMI)
Calcolo le categorie di appartenenza sulla base del valore di BMI e le inserisco in una nuova colonna del dataframe.

In [ ]:
bmi_g = []
for i in df['bmi']:
    if i > 30:
        bmi_g.append('obesità')
    elif i <= 30 and i >= 25:
        bmi_g.append('sovrappeso')
    elif i < 25 and i >= 18.5:
        bmi_g.append('normopeso')
    elif i < 18.5:
        bmi_g.append('sottopeso')

In [ ]:
df['bmi_group']=pd.Series(bmi_g)
df.sort_values('id')#.drop(columns=['index'])

### Categorie in base alla media del livello di glucosio
Calcolo le categorie di appartenenza sulla base del valore medio del livello di glucosio e le inserisco in una nuova colonna del dataframe.

In [ ]:
glucose_g = []
for i in df['avg_glucose_level']:
    if i < 60:
        glucose_g.append('ipoglicemia')
    elif i >= 60 and i <= 110:
        glucose_g.append('normale')
    elif i >= 110:
        glucose_g.append('iperglicemia')

In [ ]:
df['glucose_group']=pd.Series(glucose_g)
df.sort_values('id')

### Categorie in base all'età'
Calcolo le categorie di appartenenza sulla base dell'età e le inserisco in una nuova colonna del dataframe.

In [ ]:
age_g = []
for i in df['age']:
    if i < 35:
        age_g.append('giovane')
    elif i >= 35 and i <= 55:
        age_g.append('adulto')
    elif i >= 55:
        age_g.append('anziano')

In [ ]:
df['age_group']=pd.Series(age_g)
df.sort_values('id')

In [ ]:
# aggiorno la lista delle colonne
columnNames = df.columns.tolist()
columnNames

Ciclo for per creare tutte le coppie di immagini:

In [ ]:
for i in columnNames:
    if i != 'id' and i != 'bmi' and i != 'age' and i != 'hypertension' and i != 'avg_glucose_level':
        fig = px.histogram(
        df,x=df[str(i)],color='hypertension',title=('<b>Hypertension - '+str(i)+' relation</b>'),color_discrete_sequence=px.colors.qualitative.Pastel2
        )
        fig.show()

Non si nota un particolare comportamento prevalente per distinguere in modo evidente le due classi di ipertensione.
L'unico aspetto evidente è che l'ipertensione si presenta soprattutto nei soggetti con età maggiore.

Per vedere tutti i plot incrociati per tutti gli attributi decommentare la seguente cella:

In [ ]:
#for i in columnNames:
#    for j in columnNames:
#        if i != 'id' and i != 'bmi' and i != 'age' and j != 'id' and j != 'bmi' and j != 'age' and i != j:
#            fig = px.histogram(
#            df,x=df.bmi_group,color=str(i),title=('<b>BMI-'+str(i)+' relation</b>'),color_discrete_sequence=px.colors.qualitative.Pastel2
#            )
#        fig.show()

### Resampling
Poichè il dataset è molto sbilanciato è bene ricampionare i dati. Per evitare di sovracampionare e rendere meno realistica la previsione, si decide di utilizzare un sottogruppo di campioni che riportano un target pari a 0 e la totalità dei campioni che hanno target pari a 1:

Nuovo set di campioni con target pari a 1:

In [ ]:
resampled1 = resample(df.query('hypertension==1'), replace=True, n_samples=400, random_state=None, stratify=None)
resampled1.shape

Nuovo set di campioni con target pari a 0:

In [ ]:
resampled0 = resample(df.query('hypertension!=1'), replace=True, n_samples=1200, random_state=None, stratify=None)
resampled0.shape

Nuovo set di campioni completo:

In [ ]:
frames = [resampled0, resampled1]

resampled = pd.concat(frames)

resampled.shape

In [ ]:
# converto i valori stringa in numerici
resampled.gender = resampled.gender.replace({'Male':0,'Female':1}).astype(np.uint8)
resampled.ever_married = resampled.ever_married.replace({'No':0,'Yes':1}).astype(np.uint8)
resampled.residence_type = resampled.residence_type.replace({'Rural':0,'Urban':1}).astype(np.uint8)
resampled.work_type = resampled.work_type.replace({'Private':0,'Self-employed':1,'Govt_job':2,'children':-1,'Never_worked':-2}).astype(np.uint8)
resampled.smoking_status = resampled.smoking_status.replace({'Unknown':-1,'never smoked':0,'formerly smoked':1,'smokes':2}).astype(np.uint8)
resampled.bmi_group = resampled.bmi_group.replace({'sottopeso':-1,'normopeso':0,'sovrappeso':1,'obesità':2}).astype(np.uint8)
resampled.glucose_group = resampled.glucose_group.replace({'ipoglicemia':-1,'normale':0,'iperglicemia':1}).astype(np.uint8)
resampled.age_group = resampled.age_group.replace({'giovane':0,'adulto':1,'anziano':2}).astype(np.uint8)

resampled

Utilizzo la funzione train_test_split per dividere i campioi in train e test set, untilizzando l'attributo 'stratify' che permette di mantenere la proporzione originale di campioni con etichetta 0 o 1 (quindi 1:3 in questo caso).

## Caso 1

Caso 1:
Utilizzo le seguenti colonne di attributi per l'analisi:
- 'gender'
- 'heart_disease'
- 'ever_married'
- 'work_type'
- 'residence_type'
- 'smoking_status'
- 'stroke'
- 'bmi_group'
- 'glucose_group'
- 'age_group'

NB: non utilizzo le colonne 'bmi', 'avg_glucose_level' e 'age' che hanno dei valori puntuali ma utilizzo i loro corrispettivi gruppi 'bmi_group', 'glucose_group' e 'age_group'.

In [ ]:
X = resampled[['gender', 'heart_disease', 'ever_married', 'work_type', 'residence_type', 'smoking_status', 'stroke', 'bmi_group', 'glucose_group', 'age_group']]
y = resampled['hypertension']
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.30, random_state=1, shuffle=True, stratify=y)

### Modelli di classificazione

In [ ]:
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
models

### Risultati della classificazione

In [ ]:
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

La SVM LR e CART sono quelle che mi danno i risultati migliori.
Plotto un boxplot per comparare le performance dei modelli e i relativi risultati di predizione.

In [ ]:
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()

#### SVM

In [ ]:
model = SVC(gamma='auto')
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

In [ ]:
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

#### LR

In [ ]:
model = LogisticRegression(solver='liblinear', multi_class='ovr')
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

In [ ]:
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

#### CART

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

In [ ]:
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

## Caso 2

Caso 2:
Utilizzo le seguenti colonne di attributi per l'analisi:
- 'gender'
- 'heart_disease'
- 'ever_married'
- 'work_type'
- 'residence_type'
- 'smoking_status'
- 'stroke'
- 'bmi'
- 'avg_glucose_level'
- 'age'

NB: in questo caso utilizzo le colonne 'bmi', 'avg_glucose_level' e 'age' che hanno dei valori puntuali ed escludo i loro corrispettivi gruppi 'bmi_group', 'glucose_group' e 'age_group'.

In [ ]:
X = resampled[['gender', 'heart_disease', 'ever_married', 'work_type', 'residence_type', 'smoking_status', 'stroke', 'bmi', 'avg_glucose_level', 'age']]
y = resampled['hypertension']
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.30, random_state=1, shuffle=True, stratify=y)

### Risultati della classificazione

In [ ]:
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

Come nel caso 1, la SVM è quella con l'accuracy più alta.
Plotto un boxplot per comparare le performance dei modelli.

In [ ]:
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()

In [ ]:
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

In [ ]:
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='roc_auc')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

In [ ]:
fig = plt.figure(figsize=(12,10))
plt.title('Classification Algorithms')
plt.xlabel('Algorithm')
plt.ylabel('ROC-AUC')
plt.boxplot(results)
ax = fig.add_subplot(111)
ax.set_xticklabels(names)
plt.show()

## Commento
La confusion matrix mostra come la predizione sia molto influenzata dalla distribuzione del dataset, infatti si può notare che, in entrambi i casi, il numero più alto dovuto alle misclassificazioni è da attribuirsi al numero di falsi negativi. Questo può essere spiegato dal fatto che la parte più importante dei campioni utilizzati presenta un target negativo e quindi è più facile classificare un nuovo campione all'interno di quella classe.
La possibile differenza tra il caso 1 e il caso 2 è da attribuirsi all'utilizzo di attributi categorici rispetto a valori numerici.